In [1]:
from transformers import MarianMTModel, MarianTokenizer

model_name = 'Helsinki-NLP/opus-mt-en-zh'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Example translation
src_text = ["Dear Hiring Manager,I am writing to express my interest in the internship position at 180 Markets asadvertised. As a current Master of Information Technology student at the Universityof Melbourne with a specialization in Artificial Intelligence, I am eager to contributemy data analysis and technical skills to your esteemed organization. My educationalbackground and hands-on experience make me a strong candidate for this role, andI am excited about the opportunity to be part of a team that plays a crucial role inthe capital raising process.My academic journey has equipped me with a solid foundation in data analysis andmanagement, which I believe are critical for the responsibilities outlined in theinternship role. Courses such as Database Systems, Element of Data Processingand knowledge about statistical tools of R Studio and Matlab have provided me withthe technical expertise to efficiently handle large datasets and manage informationflow, skills that align perfectly with the requirements of maintaining and updatingyour company's master sheet and other data-related tasks."]
inputs = tokenizer(src_text, return_tensors="pt", padding=True)
translated = model.generate(**inputs)
translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)

print(translated_text)


/Users/wangcan/.pyenv/versions/3.12.2/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


亲爱的雇用经理,我写信表达我对180个市场实习职位的兴趣。作为目前墨尔本大学的信息技术硕士生,在人工智能领域具有专业专长,我渴望为贵受尊敬的组织贡献数据分析和技术技能。我的教育背景和亲身体验使我成为这一角色的强大候选人,我对有机会成为在资本筹集过程中发挥关键作用的团队的一员感到兴奋。我的学术旅程为我提供了一个坚实的数据分析和管理基础,我认为这对实习角色中概述的责任至关重要。数据库系统、数据处理要素和R Studio和Matlab的统计工具知识等课程为我提供了技术专长,以高效处理大型数据集和管理信息流,技能与维护和更新公司主页和其他数据相关任务的要求完全吻合。


In [1]:
from transformers import MarianMTModel, MarianTokenizer

# Load the model and tokenizer
model_name = 'Helsinki-NLP/opus-mt-en-zh'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Function for translating user input
def translate_text(text):
    inputs = tokenizer([text], return_tensors="pt", padding=True)  # Wrap the text in a list here
    translated = model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

# Example: Get input from the user
user_input = input("Enter text to translate: ")
output = translate_text(user_input)
print("Translated text:", output)

/Users/wangcan/.pyenv/versions/3.12.2/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Translated text: 我写信表达我对180个市场实习职位的兴趣。作为目前墨尔本大学的信息技术硕士生,我专门从事人工智能专业,我渴望为贵受尊敬的组织贡献数据分析和技术技能。我的教育背景和亲身体验使我成为这一角色的强有力候选人,我对有机会成为在资本筹集过程中发挥关键作用的团队的一员感到兴奋。我的学术旅程为我提供了数据分析和管理方面的坚实基础,我认为这对实习职责至关重要。数据库系统、数据处理要素和R Studio和Matlab的统计工具知识等课程为我提供了技术专长,以高效处理大型数据集和管理信息流动,技能与维持和更新贵公司总表和其他数据相关任务的要求完全吻合。此外,我在团队竞争方面的经验,如墨尔本Stew Pitch大学竞争和深圳科技创新竞赛,为我树立了坚实的基础。 数据库系统、数据处理要素以及R Studio和Matlab的统计工具知识,为我提供了高效处理大型数据集和管理信息流动的技术专长。 技能与维持和更新本公司总库和其他与数据相关任务的要求完全一致。此外,我在团队竞争中的经验,如墨尔本Startstew Pitch 竞争和Shencheving 竞争竞争竞争竞争竞争竞争竞争竞争对手,使我有能力在多样化的测试中有效地运用了我思考和革新能力。


In [1]:
import whisper
import sounddevice as sd
import numpy as np
import wavio
from translatepy import Translator

# Initialize Whisper model (choose a smaller model like "base" or "small" for faster performance)
model = whisper.load_model("base")

# Initialize translator
translator = Translator()

# Parameters for audio recording
samplerate = 16000
energy_threshold = 1000  # Adjust this threshold based on your environment
silence_duration = 0.5  # Duration (in seconds) of silence to stop recording

# Function to record audio until silence is detected
def record_audio_until_silence(filename, samplerate=16000):
    print("Recording...")
    audio_data = []
    silence_counter = 0
    
    with sd.InputStream(samplerate=samplerate, channels=1, dtype='int16') as stream:
        while True:
            frame = stream.read(int(samplerate * 0.1))[0]  # Read 0.1-second chunks
            audio_data.append(frame)
            
            # Calculate the energy of the current frame
            if np.max(np.abs(frame)) < energy_threshold:
                silence_counter += 0.1
            else:
                silence_counter = 0

            # If silence has been detected for a sufficient duration, stop recording
            if silence_counter >= silence_duration:
                break
    
    audio_data = np.concatenate(audio_data, axis=0)
    wavio.write(filename, audio_data, samplerate, sampwidth=2)
    print("Recording complete!")

# Continuously record and transcribe audio
while True:
    # Record audio until silence is detected and save to 'temp_audio.wav'
    record_audio_until_silence("temp_audio.wav")

    # Transcribe using Whisper
    result = model.transcribe("temp_audio.wav")
    english_text = result["text"]
    print("Recognized English:", english_text)

    # Translate to Chinese
    if english_text.strip():  # Ensure non-empty input for translation
        translation_result = translator.translate(english_text, "Chinese")
        print("Translated Chinese:", translation_result.result)

/Users/wangcan/.pyenv/versions/3.12.2/lib/python3.12/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_loc

Recording...
Recording complete!


/Users/wangcan/.pyenv/versions/3.12.2/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Recognized English: 
Recording...
Recording complete!


/Users/wangcan/.pyenv/versions/3.12.2/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


...

/Users/wangcan/.pyenv/versions/3.12.2/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Recognized English: 
Recording...
Recording complete!


/Users/wangcan/.pyenv/versions/3.12.2/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Recognized English: 
Recording...


KeyboardInterrupt: 

In [7]:
import whisper
import sounddevice as sd
import numpy as np
import wavio
from transformers import MarianMTModel, MarianTokenizer

# Initialize Whisper model (choose a smaller model like "base" or "small" for faster performance)
model = whisper.load_model("base")

# Initialize MarianMT model and tokenizer for offline translation
model_name = 'Helsinki-NLP/opus-mt-en-zh'
tokenizer = MarianTokenizer.from_pretrained(model_name)
translation_model = MarianMTModel.from_pretrained(model_name)

# Parameters for audio recording
samplerate = 16000
energy_threshold = 1000  # Adjust this threshold based on your environment
silence_duration = 0.5  # Duration (in seconds) of silence to stop recording

show = 1

# Function to record audio until silence is detected
def record_audio_until_silence(filename, samplerate=16000):
    if show:
        print("Recording...")
    audio_data = []
    silence_counter = 0
    
    with sd.InputStream(samplerate=samplerate, channels=1, dtype='int16') as stream:
        while True:
            frame = stream.read(int(samplerate * 0.1))[0]  # Read 0.1-second chunks
            audio_data.append(frame)
            
            # Calculate the energy of the current frame
            if np.max(np.abs(frame)) < energy_threshold:
                silence_counter += 0.1
            else:
                silence_counter = 0

            # If silence has been detected for a sufficient duration, stop recording
            if silence_counter >= silence_duration:
                break
    
    audio_data = np.concatenate(audio_data, axis=0)
    wavio.write(filename, audio_data, samplerate, sampwidth=2)
    if show:
        print("Recording complete!")

# Function to translate English text to Chinese using the MarianMT model
def translate_to_chinese(text):
    inputs = tokenizer([text], return_tensors="pt", padding=True)
    translated = translation_model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

# Continuously record and transcribe audio
while True:
    # Record audio until silence is detected and save to 'temp_audio.wav'
    record_audio_until_silence("temp_audio.wav")

    # Transcribe using Whisper
    result = model.transcribe("temp_audio.wav")
    english_text = result["text"]
    # only print the recognized text if it is not empty
    if english_text:
        print("Recognized English:", english_text)

    # Translate to Chinese
    if english_text.strip():  # Ensure non-empty input for translation
        chinese_text = translate_to_chinese(english_text)
        print("Translated Chinese:", chinese_text)


/Users/wangcan/.pyenv/versions/3.12.2/lib/python3.12/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_loc

Recording...
Recording complete!


/Users/wangcan/.pyenv/versions/3.12.2/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Recognized English: 13歲了
Translated Chinese: 13 {_____________________________________________________________________________________________________________________________
Recording...
Recording complete!


/Users/wangcan/.pyenv/versions/3.12.2/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Recording...


KeyboardInterrupt: 

In [12]:
import warnings
import whisper
import sounddevice as sd
import numpy as np
import wavio
from transformers import MarianMTModel, MarianTokenizer
import threading
import queue

# Switch to show recording status
show = False

if show:
    warnings.filterwarnings("ignore")

# Initialize Whisper model (choose a smaller model like "base" or "small" for faster performance)
model = whisper.load_model("small")

# Initialize MarianMT model and tokenizer for offline translation
model_name = 'Helsinki-NLP/opus-mt-en-zh'
tokenizer = MarianTokenizer.from_pretrained(model_name)
translation_model = MarianMTModel.from_pretrained(model_name)

# Parameters for audio recording
samplerate = 16000
energy_threshold = 1000  # Adjust this threshold based on your environment
silence_duration = 0.5  # Duration (in seconds) of silence to stop recording

# Queue to hold audio files to be processed
audio_queue = queue.Queue()

# Function to record audio until silence is detected
def record_audio_until_silence(filename, samplerate=16000):
    if show:
        print("Recording...")
    audio_data = []
    silence_counter = 0
    
    with sd.InputStream(samplerate=samplerate, channels=1, dtype='int16') as stream:
        while True:
            frame = stream.read(int(samplerate * 0.1))[0]  # Read 0.1-second chunks
            audio_data.append(frame)
            
            # Calculate the energy of the current frame
            if np.max(np.abs(frame)) < energy_threshold:
                silence_counter += 0.1
            else:
                silence_counter = 0

            # If silence has been detected for a sufficient duration, stop recording
            if silence_counter >= silence_duration:
                break
    
    audio_data = np.concatenate(audio_data, axis=0)
    wavio.write(filename, audio_data, samplerate, sampwidth=2)
    if show:
        print("Recording complete!")
    audio_queue.put(filename)  # Add the recorded audio to the processing queue

# Function to translate English text to Chinese using the MarianMT model
def translate_to_chinese(text):
    inputs = tokenizer([text], return_tensors="pt", padding=True)
    translated = translation_model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

# Function to process audio (transcribe and translate)
def process_audio():
    while True:
        filename = audio_queue.get()  # Get the next audio file from the queue
        if filename:
            # Transcribe using Whisper
            result = model.transcribe(filename, fp16=False)
            english_text = result["text"]
            # Only print the recognized text if it is not empty
            if english_text:
                print("Recognized English:", english_text)

                # Translate to Chinese
                if english_text.strip():  # Ensure non-empty input for translation
                    chinese_text = translate_to_chinese(english_text)
                    print("Translated Chinese:", chinese_text)
        
        audio_queue.task_done()  # Mark the task as done

# Start a thread for continuous audio processing
threading.Thread(target=process_audio, daemon=True).start()

# Continuously record audio until stopped
while True:
    # Record audio and save to a temporary file
    record_audio_until_silence("temp_audio.wav")


100%|███████████████████████████████████████| 461M/461M [00:59<00:00, 8.10MiB/s]
/Users/wangcan/.pyenv/versions/3.12.2/lib/python3.12/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any i

Recognized English:  what my goal for the subject for all of you is, I'm going to turn to the professor who teaches Harvard CS50, Professor David. And as you'll see in the coming weeks, as you write your own code and solve your own problems, what ultimately matters in this course is not so much that you're not just a teacher.
Translated Chinese: 我对你们所有人来说的目标是什么, 我要向教授哈佛CS50的教授,大卫教授。正如你们今后几周将看到的那样, 当你写出自己的代码,解决自己的问题时, 最终重要的是, 在这个过程中,你不只是个老师。
Recognized English:  where you end up relative to your classmates, but where you end up relative to yourself when you began. And it really is all about that delta with you.
Translated Chinese: 在那里,你最终与你的同学相对, 但当你开始的时候,你最终与你自己相对。 这其实都是关于与你同在的三角洲。
Recognized English:  or not just
Translated Chinese: 或不仅仅是
Recognized English:  something out of a class like this and if it does take time
Translated Chinese: 像这样的班级里的东西 如果需要时间的话
Recognized English:  you do feel those frustrations, but you simultaneously eventually feel that sense of accomplishment.
Transla

KeyboardInterrupt: 

In [ ]:
import warnings
import whisper
import sounddevice as sd
import numpy as np
import wavio
from transformers import MarianMTModel, MarianTokenizer
import threading
import queue

# Switch to show recording status
show = False

if show:
    warnings.filterwarnings("ignore")

# Initialize Whisper model (choose a smaller model like "base" or "small" for faster performance)
model = whisper.load_model("small")

# Initialize MarianMT model and tokenizer for offline translation
model_name = 'Helsinki-NLP/opus-mt-en-zh'
tokenizer = MarianTokenizer.from_pretrained(model_name)
translation_model = MarianMTModel.from_pretrained(model_name)

# Parameters for audio recording
samplerate = 16000
energy_threshold = 1000  # Adjust this threshold based on your environment
silence_duration = 0.5  # Duration (in seconds) of silence to stop recording

# Queue to hold audio files to be processed
audio_queue = queue.Queue()

# Path for the combined script file
combined_script_path = "script.txt"

# Function to record audio until silence is detected
def record_audio_until_silence(filename, samplerate=16000):
    if show:
        print("Recording...")
    audio_data = []
    silence_counter = 0
    
    with sd.InputStream(samplerate=samplerate, channels=1, dtype='int16') as stream:
        while True:
            frame = stream.read(int(samplerate * 0.1))[0]  # Read 0.1-second chunks
            audio_data.append(frame)
            
            # Calculate the energy of the current frame
            if np.max(np.abs(frame)) < energy_threshold:
                silence_counter += 0.1
            else:
                silence_counter = 0

            # If silence has been detected for a sufficient duration, stop recording
            if silence_counter >= silence_duration:
                break
    
    audio_data = np.concatenate(audio_data, axis=0)
    wavio.write(filename, audio_data, samplerate, sampwidth=2)
    if show:
        print("Recording complete!")
    audio_queue.put(filename)  # Add the recorded audio to the processing queue

# Function to translate English text to Chinese using the MarianMT model
def translate_to_chinese(text):
    inputs = tokenizer([text], return_tensors="pt", padding=True)
    translated = translation_model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

# Function to process audio (transcribe and translate)
def process_audio():
    while True:
        filename = audio_queue.get()  # Get the next audio file from the queue
        if filename:
            # Transcribe using Whisper
            result = model.transcribe(filename, fp16=False)
            english_text = result["text"]
            # Only print the recognized text if it is not empty
            if english_text:
                print("Recognized English:", english_text)

                # Translate to Chinese
                if english_text.strip():  # Ensure non-empty input for translation
                    chinese_text = translate_to_chinese(english_text)
                    print("Translated Chinese:", chinese_text)

                    # Write both English and Chinese to the combined script file
                    with open(combined_script_path, "a") as combined_file:
                        combined_file.write("English: " + english_text + "\n")
                        combined_file.write("Chinese: " + chinese_text + "\n")
                        combined_file.write("\n")  # Add a blank line for separation
        
        audio_queue.task_done()  # Mark the task as done

# Start a thread for continuous audio processing
threading.Thread(target=process_audio, daemon=True).start()

# Continuously record audio until stopped
while True:
    # Record audio and save to a temporary file
    record_audio_until_silence("temp_audio.wav")


/Users/wangcan/.pyenv/versions/3.12.2/lib/python3.12/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_loc

Recognized English:  If you've given your number away to someone...
Translated Chinese: 如果你把号码给别人了...
Recognized English:  3, 2, 1.
Translated Chinese: 3, 2, 1, 3, 3, 1.
Recognized English:  You should be sitting down now.
Translated Chinese: 你该坐下了
Recognized English:  You should be sitting down now.
Translated Chinese: 你该坐下了
Recognized English:  By Dan
Translated Chinese: 丹丹
Recognized English:  51, okay? Add 51 to whatever you got.
Translated Chinese: 51,好吗? 随你多多加51
Recognized English:  Okay.
Translated Chinese: 好吧。 好吧。 好吧。 Okay.
Recognized English:  Got it?
Translated Chinese: 明白了吗?
Recognized English:  3
Translated Chinese: 3, 3, 3, 3
Recognized English:  Okay, how many people are in the room?
Translated Chinese: 好吧 房间里有多少人?


KeyboardInterrupt: 

In [14]:
import warnings
import whisper
import sounddevice as sd
import numpy as np
import wavio
from transformers import MarianMTModel, MarianTokenizer
import threading
import queue
import datetime

# Switch to show recording status
show = 0

if not show:
    warnings.filterwarnings("ignore")

# Initialize Whisper model (choose a smaller model like "base" or "small" for faster performance)
model = whisper.load_model("small")

# Initialize MarianMT model and tokenizer for offline translation
model_name = 'Helsinki-NLP/opus-mt-en-zh'
tokenizer = MarianTokenizer.from_pretrained(model_name)
translation_model = MarianMTModel.from_pretrained(model_name)

# Parameters for audio recording
samplerate = 16000
energy_threshold = 1000  # Adjust this threshold based on your environment
silence_duration = 0.5  # Duration (in seconds) of silence to stop recording

# Queue to hold audio files to be processed
audio_queue = queue.Queue()

# Path for the combined script file, name file with current_time saved when start listening
combined_script_path = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + ".txt"

# Variable to store the last processed English text
last_processed_text = None

# Function to record audio until silence is detected
def record_audio_until_silence(filename, samplerate=16000):
    if show:
        print("Recording...")
    audio_data = []
    silence_counter = 0
    
    with sd.InputStream(samplerate=samplerate, channels=1, dtype='int16') as stream:
        while True:
            frame = stream.read(int(samplerate * 0.1))[0]  # Read 0.1-second chunks
            audio_data.append(frame)
            
            # Calculate the energy of the current frame
            if np.max(np.abs(frame)) < energy_threshold:
                silence_counter += 0.1
            else:
                silence_counter = 0

            # If silence has been detected for a sufficient duration, stop recording
            if silence_counter >= silence_duration:
                break
    
    audio_data = np.concatenate(audio_data, axis=0)
    wavio.write(filename, audio_data, samplerate, sampwidth=2)
    if show:
        print("Recording complete!")
    audio_queue.put(filename)  # Add the recorded audio to the processing queue

# Function to translate English text to Chinese using the MarianMT model
def translate_to_chinese(text):
    inputs = tokenizer([text], return_tensors="pt", padding=True)
    translated = translation_model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

# Function to process audio (transcribe and translate)
def process_audio():
    global last_processed_text
    while True:
        filename = audio_queue.get()  # Get the next audio file from the queue
        if filename:
            # Transcribe using Whisper
            result = model.transcribe(filename, fp16=False)
            english_text = result["text"].strip()
            
            # Only process if the new text is different from the last processed text
            if english_text and english_text != last_processed_text:
                print("Recognized English:", english_text)
                last_processed_text = english_text  # Update the last processed text

                # Translate to Chinese
                chinese_text = translate_to_chinese(english_text)
                print("Translated Chinese:", chinese_text)

                # Write both English and Chinese to the combined script file
                with open(combined_script_path, "a") as combined_file:
                    combined_file.write("English: " + english_text + "\n")
                    combined_file.write("Chinese: " + chinese_text + "\n")
                    combined_file.write("\n")  # Add a blank line for separation
        
        audio_queue.task_done()  # Mark the task as done

current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print("Recording started at", current_time)
# Start a thread for continuous audio processing
threading.Thread(target=process_audio, daemon=True).start()

# Continuously record audio until stopped
while True:
    # Record audio and save to a temporary file
    record_audio_until_silence("temp_audio.wav")


Recording started at 2024-11-11 23:39:52
Recognized English: Hello everyone and again welcome to Foundations of the Algorathans for the last session of Week 1.
Translated Chinese: 大家好,欢迎再次来到阿尔哥拉坦基金会 参加第1周的最后一场活动
Recognized English: So, as you'll remember, at the end of the last lecture, we were talking a bit about how to get stunned and see.
Translated Chinese: 所以,你们会记得,在最后一场演讲结束时, 我们谈论了如何被惊吓和看一看。
Recognized English: But unlike in other programming languages that you might have seen before, we were fussing a lot over the types of data that we were actually feeding into our program. Whether it was a plain number or whether it was a number of a decimal point.
Translated Chinese: 但与你以前可能见过的其他编程语言不同的是,我们在编程中实际输入的数据类型上大惊小怪。它是一个简单的数字,还是一个小数点的数字。
Recognized English: as we soon discover and see where actually going to be required to always specify.
Translated Chinese: 当我们很快发现 并看看到底在哪里需要 总是具体说明。
Recognized English: kind of information when getting the computer to deal with.
Translated Chinese: 

KeyboardInterrupt: 

In [8]:
import dearpygui.dearpygui as dpg

def button_callback(sender, app_data, user_data):
    dpg.set_value("label", "Hello, World!")

dpg.create_context()

with dpg.window(label="Main Window"):
    dpg.add_text("Click button")
    dpg.add_button(label="Click here", callback=button_callback)
    dpg.add_text("Hello, ", tag="label")

dpg.create_viewport(title='Dear PyGui demo', width=600, height=300)
dpg.setup_dearpygui()
dpg.show_viewport()
dpg.start_dearpygui()
dpg.destroy_context()


In [2]:
import dearpygui.dearpygui as dpg

# Create a Dear PyGui context
dpg.create_context()

# Create a window with an input text box
with dpg.window(label="Text Box Example", width=400, height=300):
    dpg.add_text("Please enter your text below:")
    dpg.add_input_text(label="Plain Text Box", multiline=True, width=350, height=150)

# Create viewport and setup
dpg.create_viewport(title="Plain Text Box Example", width=600, height=400)
dpg.setup_dearpygui()
dpg.show_viewport()
dpg.start_dearpygui()
dpg.destroy_context()


2024-11-11 17:21:20.114 python[72376:20805944] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


In [2]:
import tkinter as tk
from tkinter import ttk

# Create the main application window
root = tk.Tk()
root.title("Voice Recognition and Translation App")
root.geometry("800x600")  # Set the window size

# Create a notebook (tabbed interface)
notebook = ttk.Notebook(root)

# Create frames for each tab
voice_recognition_tab = ttk.Frame(notebook)
translation_tab = ttk.Frame(notebook)

# Add the tabs to the notebook
notebook.add(voice_recognition_tab, text="Voice Recognition")
notebook.add(translation_tab, text="Translation")

# Pack the notebook to fill the entire window
notebook.pack(expand=True, fill="both")

# Define the layout for the Voice Recognition tab
voice_left_frame = tk.Frame(voice_recognition_tab, bg="lightblue", width=400, height=600)
voice_right_frame = tk.Frame(voice_recognition_tab, bg="lightgreen", width=400, height=600)

voice_left_frame.pack(side="left", expand=True, fill="both")
voice_right_frame.pack(side="right", expand=True, fill="both")

voice_left_label = tk.Label(voice_left_frame, text="Voice Input", font=("Helvetica", 16))
voice_left_label.pack(pady=10)
voice_right_label = tk.Label(voice_right_frame, text="Processing Results", font=("Helvetica", 16))
voice_right_label.pack(pady=10)

# Define the layout for the Translation tab
translation_left_frame = tk.Frame(translation_tab, bg="lightyellow", width=400, height=600)
translation_right_frame = tk.Frame(translation_tab, bg="lightpink", width=400, height=600)

translation_left_frame.pack(side="left", expand=True, fill="both")
translation_right_frame.pack(side="right", expand=True, fill="both")

translation_left_label = tk.Label(translation_left_frame, text="Text Input", font=("Helvetica", 16))
translation_left_label.pack(pady=10)
translation_right_label = tk.Label(translation_right_frame, text="Translation Results", font=("Helvetica", 16))
translation_right_label.pack(pady=10)

# Start the Tkinter event loop
root.mainloop()


2024-11-11 17:25:45.458 Python[72811:20815874] _TIPropertyValueIsValid called with 11 on nil context!
2024-11-11 17:25:45.458 Python[72811:20815874] imkxpc_setApplicationProperty:value:reply: called with incorrect property value 11, bailing.
2024-11-11 17:25:45.458 Python[72811:20815874] _TIPropertyValueIsValid called with 12 on nil context!
2024-11-11 17:25:45.458 Python[72811:20815874] imkxpc_setApplicationProperty:value:reply: called with incorrect property value 12, bailing.
2024-11-11 17:25:45.562 Python[72811:20815874] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.
2024-11-11 17:25:46.982 Python[72811:20815874] _TIPropertyValueIsValid called with 11 on nil context!
2024-11-11 17:25:46.982 Python[72811:20815874] imkxpc_setApplicationProperty:value:reply: called with incorrect pro

In [11]:
import warnings
import whisper
import sounddevice as sd
import numpy as np
import wavio
import threading
import queue
import datetime

# Switch to show recording status
show = 0

if not show:
    warnings.filterwarnings("ignore")

# Initialize Whisper model (choose a smaller model like "base" or "small" for faster performance)
model = whisper.load_model("small")

# Parameters for audio recording
samplerate = 16000
energy_threshold = 1000  # Adjust this threshold based on your environment
silence_duration = 0.5  # Duration (in seconds) of silence to stop recording

# Queue to hold audio files to be processed
audio_queue = queue.Queue()

# Path for the combined script file, name file with current_time saved when start listening
combined_script_path = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + ".txt"

# Variable to store the last processed English text
last_processed_text = None

# Function to record audio until silence is detected
def record_audio_until_silence(filename, samplerate=16000):
    if show:
        print("Recording...")
    audio_data = []
    silence_counter = 0
    
    with sd.InputStream(samplerate=samplerate, channels=1, dtype='int16') as stream:
        while True:
            frame = stream.read(int(samplerate * 0.1))[0]  # Read 0.1-second chunks
            audio_data.append(frame)
            
            # Calculate the energy of the current frame
            if np.max(np.abs(frame)) < energy_threshold:
                silence_counter += 0.1
            else:
                silence_counter = 0

            # If silence has been detected for a sufficient duration, stop recording
            if silence_counter >= silence_duration:
                break
    
    audio_data = np.concatenate(audio_data, axis=0)
    wavio.write(filename, audio_data, samplerate, sampwidth=2)
    if show:
        print("Recording complete!")
    audio_queue.put(filename)  # Add the recorded audio to the processing queue

# Function to process audio (transcribe and translate using Whisper)
def process_audio():
    global last_processed_text
    while True:
        filename = audio_queue.get()  # Get the next audio file from the queue
        if filename:
            # Transcribe using Whisper and translate directly to Chinese
            result = model.transcribe(filename, language="zh", fp16=False)
            translated_text = result["text"].strip()

            # Only process if the new text is different from the last processed text
            if translated_text and translated_text != last_processed_text:
                print("Translated Chinese:", translated_text)
                last_processed_text = translated_text  # Update the last processed text

                # Write the translated text to the combined script file
                with open(combined_script_path, "a") as combined_file:
                    combined_file.write("Chinese: " + translated_text + "\n")
                    combined_file.write("\n")  # Add a blank line for separation
        
        audio_queue.task_done()  # Mark the task as done

current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print("Recording started at", current_time)
# Start a thread for continuous audio processing
threading.Thread(target=process_audio, daemon=True).start()

# Continuously record audio until stopped
while True:
    # Record audio and save to a temporary file
    record_audio_until_silence("temp_audio.wav")


Recording started at 2024-11-11 23:30:54
Translated Chinese: 哈啰今天是11月2024
Translated Chinese: 现在重 broadcast鱼
Translated Chinese: 英文声音
Translated Chinese: 然而称为 sansloveózادffen달虗米清呀我欧阳前方涛歐他这里
Translated Chinese: 字幕by索兰娅


KeyboardInterrupt: 

In [10]:
import warnings
import whisper
import sounddevice as sd
import numpy as np
import wavio
from transformers import MarianMTModel, MarianTokenizer
import threading
import queue
import datetime
import os

# Switch to show recording status
show = 0

if not show:
    warnings.filterwarnings("ignore")

# Initialize Whisper model (choose a smaller model like "base" or "small" for faster performance)
# model = whisper.load_model("tiny.en")
model = whisper.load_model("small")

# Initialize MarianMT model and tokenizer for offline translation
model_name = 'Helsinki-NLP/opus-mt-en-zh'
tokenizer = MarianTokenizer.from_pretrained(model_name)
translation_model = MarianMTModel.from_pretrained(model_name)

# Initialize MarianMT model and tokenizer again for offline chinese to english translation
model_name_chinese = 'Helsinki-NLP/opus-mt-zh-en'
tokenizer_chinese = MarianTokenizer.from_pretrained(model_name_chinese)
translation_model_chinese = MarianMTModel.from_pretrained(model_name_chinese)

# Parameters for audio recording
samplerate = 16000
energy_threshold = 1000  # Adjust this threshold based on your environment
silence_duration = 0.4  # Duration (in seconds) of silence to stop recording

# Queue to hold audio files to be processed
audio_queue = queue.Queue()

# Path for the combined script file, name file with current_time saved when start listening
combined_script_path = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + ".txt"

# Variable to store the last processed English text
last_processed_text = None

# Function to record audio until silence is detected
def record_audio_until_silence(samplerate=16000):
    if show:
        print("Recording...")
    audio_data = []
    silence_counter = 0
    
    with sd.InputStream(samplerate=samplerate, channels=1, dtype='int16') as stream:
        while True:
            frame = stream.read(int(samplerate * 0.1))[0]  # Read 0.1-second chunks
            audio_data.append(frame)
            
            # Calculate the energy of the current frame
            if np.max(np.abs(frame)) < energy_threshold:
                silence_counter += 0.1
            else:
                silence_counter = 0

            # If silence has been detected for a sufficient duration, stop recording
            if silence_counter >= silence_duration:
                break
    
    audio_data = np.concatenate(audio_data, axis=0)
    # Create a timestamped filename for the audio file
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"audio_{timestamp}.wav"
    wavio.write(filename, audio_data, samplerate, sampwidth=2)
    if show:
        print("Recording complete!")
    audio_queue.put(filename)  # Add the recorded audio to the processing queue

# Function to translate English text to Chinese using the MarianMT model
def translate_to_chinese(text):
    inputs = tokenizer([text], return_tensors="pt", padding=True)
    translated = translation_model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

# Function to translate Chinese text to English using the MarianMT model
def translate_to_english(text):
    inputs = tokenizer_chinese([text], return_tensors="pt", padding=True)
    translated = translation_model_chinese.generate(**inputs)
    translated_text = tokenizer_chinese.decode(translated[0], skip_special_tokens=True)
    return translated

# Function to process audio (transcribe and translate)
def process_audio():
    global last_processed_text
    while True:
        filename = audio_queue.get()  # Get the next audio file from the queue
        if filename:
            # Transcribe using Whisper
            result = model.transcribe(filename, fp16=False)
            english_text = result["text"].strip()
            
            # Only process if the new text is different from the last processed text
            if english_text and english_text != last_processed_text:
                print("Recognized English:", english_text)
                last_processed_text = english_text  # Update the last processed text

                # Translate to Chinese
                chinese_text = translate_to_chinese(english_text)
                print("Translated Chinese:", chinese_text)

                # Write both English and Chinese to the combined script file
                with open(combined_script_path, "a") as combined_file:
                    combined_file.write("English: " + english_text + "\n")
                    combined_file.write("Chinese: " + chinese_text + "\n")
                    combined_file.write("\n")  # Add a blank line for separation

            # Delete the processed audio file
            if os.path.exists(filename):
                os.remove(filename)
                if show:
                    print(f"Deleted file: {filename}")
        
        audio_queue.task_done()  # Mark the task as done

current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print("Recording started at", current_time)

# Start a thread for continuous audio processing
threading.Thread(target=process_audio, daemon=True).start()

# Continuously record audio until stopped
while True:
    # Record audio and save to a temporary file
    record_audio_until_silence()


Recording started at 2024-11-21 13:31:21
Recognized English: Hello everyone and again welcome to Foundations of Algoradians for the last session of week one.
Translated Chinese: 大家好,再次欢迎来到阿尔戈拉迪安基金会 参加第一周的最后一场会议
Recognized English: To remember at the end of the last lecture, we were talking a bit about how to get stung and see.
Translated Chinese: 记得上次演讲结束时,我们谈论了 如何被刺伤和看到。
Recognized English: Unlike in other programming languages that you might have seen before, we were fussing a lot over the types of data that we were actually feeding into our program. Whether it was a plain number or whether it was a number of a decimal point.
Translated Chinese: 与你们以前可能看到的其他编程语言不同,我们一直在大肆讨论我们实际输入到我们程序的数据类型。它是一个简单的数字,还是一个十进制点。
Recognized English: I will soon discover and see where it actually going to be required to always specify
Translated Chinese: 我很快会发现 并看看到底在哪里 需要它总是指明
Recognized English: the information when you get to deal with it.
Translated Chinese: 当你得到处理时的信息。
Recognized English: Why would w

Exception in thread Thread-25 (process_audio):
Traceback (most recent call last):
  File "/Users/wangcan/.pyenv/versions/3.12.2/lib/python3.12/site-packages/whisper/audio.py", line 58, in load_audio
    out = run(cmd, capture_output=True, check=True).stdout
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wangcan/.pyenv/versions/3.12.2/lib/python3.12/subprocess.py", line 571, in run
    raise CalledProcessError(retcode, process.args,
subprocess.CalledProcessError: Command '['ffmpeg', '-nostdin', '-threads', '0', '-i', 'audio_2024-11-21_13-32-15.wav', '-f', 's16le', '-ac', '1', '-acodec', 'pcm_s16le', '-ar', '16000', '-']' returned non-zero exit status 254.

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/wangcan/.pyenv/versions/3.12.2/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/Users/wangcan/.pyenv/versions/3.12.2/lib/python3.12/site-packages/ipykernel/ipke

KeyboardInterrupt: 

In [12]:
import warnings
import whisper
import sounddevice as sd
import numpy as np
import wavio
from transformers import MarianMTModel, MarianTokenizer
import threading
import queue
import datetime
import os

# Switch to show recording status
show = 1

if not show:
    warnings.filterwarnings("ignore")

# Initialize Whisper model (choose a smaller model like "base" or "small" for faster performance)
# model = whisper.load_model("tiny.en")
model = whisper.load_model("small")

# Initialize MarianMT model and tokenizer for offline translation
model_name = 'Helsinki-NLP/opus-mt-en-zh'
tokenizer = MarianTokenizer.from_pretrained(model_name)
translation_model = MarianMTModel.from_pretrained(model_name)

# Initialize MarianMT model and tokenizer again for offline chinese to english translation
model_name_chinese = 'Helsinki-NLP/opus-mt-zh-en'
tokenizer_chinese = MarianTokenizer.from_pretrained(model_name_chinese)
translation_model_chinese = MarianMTModel.from_pretrained(model_name_chinese)

# Parameters for audio recording
samplerate = 16000
energy_threshold = 1000  # Adjust this threshold based on your environment
silence_duration = 0.4  # Duration (in seconds) of silence to stop recording

# Queue to hold audio files to be processed
audio_queue = queue.Queue()

# Path for the combined script file, name file with current_time saved when start listening
combined_script_path = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + ".txt"

# Variable to store the last processed English text
last_processed_text = None

# Function to record audio until silence is detected
def record_audio_until_silence(samplerate=16000):
    if show:
        print("Recording...")
    audio_data = []
    silence_counter = 0
    
    with sd.InputStream(samplerate=samplerate, channels=1, dtype='int16') as stream:
        while True:
            frame = stream.read(int(samplerate * 0.1))[0]  # Read 0.1-second chunks
            audio_data.append(frame)
            
            # Calculate the energy of the current frame
            if np.max(np.abs(frame)) < energy_threshold:
                silence_counter += 0.1
            else:
                silence_counter = 0

            # If silence has been detected for a sufficient duration, stop recording
            if silence_counter >= silence_duration:
                break
    
    audio_data = np.concatenate(audio_data, axis=0)
    # Create a timestamped filename for the audio file
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"audio_{timestamp}.wav"
    wavio.write(filename, audio_data, samplerate, sampwidth=2)
    if show:
        print("Recording complete!")
    audio_queue.put(filename)  # Add the recorded audio to the processing queue

# Function to translate English text to Chinese using the MarianMT model
def translate_to_chinese(text):
    inputs = tokenizer([text], return_tensors="pt", padding=True)
    translated = translation_model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

# Function to translate Chinese text to English using the MarianMT model
def translate_to_english(text):
    inputs = tokenizer_chinese([text], return_tensors="pt", padding=True)
    translated = translation_model_chinese.generate(**inputs)
    translated_text = tokenizer_chinese.decode(translated[0], skip_special_tokens=True)
    return translated

# Function to process audio (transcribe and translate)
# def process_audio():
#     global last_processed_text
#     while True:
#         filename = audio_queue.get()  # Get the next audio file from the queue
#         if filename:
#             # Transcribe using Whisper
#             result = model.transcribe(filename, fp16=False)
#             english_text = result["text"].strip()
            
#             # Only process if the new text is different from the last processed text
#             if english_text and english_text != last_processed_text:
#                 print("Recognized English:", english_text)
#                 last_processed_text = english_text  # Update the last processed text

#                 # Translate to Chinese
#                 chinese_text = translate_to_chinese(english_text)
#                 print("Translated Chinese:", chinese_text)

#                 # Write both English and Chinese to the combined script file
#                 with open(combined_script_path, "a") as combined_file:
#                     combined_file.write("English: " + english_text + "\n")
#                     combined_file.write("Chinese: " + chinese_text + "\n")
#                     combined_file.write("\n")  # Add a blank line for separation

#             # Delete the processed audio file
#             if os.path.exists(filename):
#                 os.remove(filename)
#                 if show:
#                     print(f"Deleted file: {filename}")
        
#         audio_queue.task_done()  # Mark the task as done

def process_audio():
    global last_processed_text
    while True:
        filename = audio_queue.get()  # Get the next audio file from the queue
        if filename:
            try:
                if show:
                    print(f"Processing file: {filename}")
                
                # Check if the file exists
                if not os.path.exists(filename):
                    if show:
                        print(f"Error: File {filename} does not exist. Skipping.")
                    audio_queue.task_done()
                    continue

                # Transcribe the audio
                result = model.transcribe(filename, fp16=False)
                english_text = result["text"].strip()

                # Only process if the text is different from the last processed text
                if english_text and english_text != last_processed_text:
                    print("Recognized English:", english_text)
                    last_processed_text = english_text

                    # Translate the text
                    chinese_text = translate_to_chinese(english_text)
                    print("Translated Chinese:", chinese_text)

                    # Write to the combined script file
                    with open(combined_script_path, "a") as combined_file:
                        combined_file.write(f"English: {english_text}\n")
                        combined_file.write(f"Chinese: {chinese_text}\n\n")

            except Exception as e:
                # Log the error and continue
                if show:
                    print(f"Error processing file {filename}: {e}")

            finally:
                # Delete the file and mark the task as done
                if os.path.exists(filename):
                    os.remove(filename)
                    if show:
                        print(f"Deleted file: {filename}")
                audio_queue.task_done()


current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print("Recording started at", current_time)

# Start a thread for continuous audio processing
threading.Thread(target=process_audio, daemon=True).start()

# Continuously record audio until stopped
while True:
    # Record audio and save to a temporary file
    record_audio_until_silence()


Recording started at 2024-11-21 13:47:36
Recording...
Recording complete!
Recording...
Processing file: audio_2024-11-21_13-47-37.wav
Recording complete!
Recording...
Recording complete!
Recording...
Recording complete!
Recording...
Deleted file: audio_2024-11-21_13-47-37.wav
Processing file: audio_2024-11-21_13-47-38.wav
Recording complete!
Recording...
Deleted file: audio_2024-11-21_13-47-38.wav
Processing file: audio_2024-11-21_13-47-38.wav
Error: File audio_2024-11-21_13-47-38.wav does not exist. Skipping.
Processing file: audio_2024-11-21_13-47-39.wav
Deleted file: audio_2024-11-21_13-47-39.wav
Processing file: audio_2024-11-21_13-47-40.wav


Exception in thread Thread-28 (process_audio):
Traceback (most recent call last):
  File "/Users/wangcan/.pyenv/versions/3.12.2/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/Users/wangcan/.pyenv/versions/3.12.2/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Users/wangcan/.pyenv/versions/3.12.2/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/05/7l6q95kx0cn9wtb_lc_3cz3h0000gn/T/ipykernel_35875/2414492099.py", line 169, in process_audio
  File "/Users/wangcan/.pyenv/versions/3.12.2/lib/python3.12/queue.py", line 75, in task_done
    raise ValueError('task_done() called too many times')
ValueError: task_done() called too many times


Deleted file: audio_2024-11-21_13-47-40.wav
Recording complete!
Recording...
Recording complete!
Recording...
Recording complete!
Recording...
Recording complete!
Recording...
Recording complete!
Recording...
Recording complete!
Recording...
Recording complete!
Recording...
Recording complete!
Recording...
Recording complete!
Recording...
Recording complete!
Recording...
Recording complete!
Recording...
Recording complete!
Recording...
Recording complete!
Recording...
Recording complete!
Recording...
Recording complete!
Recording...
Recording complete!
Recording...
Recording complete!
Recording...
Recording complete!
Recording...


KeyboardInterrupt: 

In [ ]:
from RealtimeSTT import AudioToTextRecorder

# 初始化音频到文本录制器
recorder = AudioToTextRecorder(
    model='tiny.en',  # 指定 Whisper 模型
    device='cpu'      # 设置设备为 CPU
)

# 设置实时转录的回调函数
def transcription_callback(transcription_text):
    print(f"实时转录: {transcription_text}")

# 配置实时转录回调
recorder.on_realtime_transcription_update = transcription_callback

# 开始录音
recorder.start()

# 等待用户输入停止
input("按回车键停止转录...\n")

# 停止录音
recorder.stop()


⠋ recording

[2024-11-21 20:34:27.620] [ctranslate2] [thread 27523080] [warning] The compute type inferred from the saved model is float16, but the target device or backend do not support efficient float16 computation. The model weights have been automatically converted to use the float32 compute type instead.


⠙ recording⠹ recording⠸ recording⠼ recording⠴ recording⠦ recording⠧ recording⠇ recording⠏ recording⠋ recording⠙ recording⠹ recording⠸ recording⠼ recording⠴ recording⠦ recording⠧ recording⠇ recording⠏ recording⠋ recording⠙ recording⠹ recording⠸ recording⠼ recording⠴ recording⠦ recording⠧ recording⠇ recording⠏ recording⠋ recording⠙ recording⠹ recording⠸ recording⠼ recording⠴ recording⠦ recording⠧ recording⠇ recording⠏ recording⠋ recording⠙ recording⠹ recording⠸ recording⠼ recording⠴ recording⠦ recording⠧ recording⠇ recording⠏ recording⠋ recording⠙ recording⠹ recording⠸ recording⠼ recording⠴ recording⠦ recording⠧ recording⠇ recording⠏ recording⠋ recording⠙ recording⠹ recording⠸ recording⠼ recording⠴ recording⠦ recording⠧ recording⠇ recording⠏ recording⠋ recording⠙ recording⠹ recording⠸ recording⠼ recording⠴ recording⠦ recording⠧ recording⠇ recording⠏ recording⠋ recording⠙ recording⠹ recording⠸ recording⠼ recording⠴ recording⠦ recording⠧ recording⠇ recording⠏ recording⠋ recording⠙ recordin

Error in callback <function Halo.__init__.<locals>.clean_up at 0x173798cc0> (for post_run_cell), with arguments args (<ExecutionResult object at 1732789b0, execution_count=10 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 173279670, raw_cell="from RealtimeSTT import AudioToTextRecorder

# 初始化.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/wangcan/Desktop/VTT/test.ipynb#X20sZmlsZQ%3D%3D> result=<RealtimeSTT.audio_recorder.AudioToTextRecorder object at 0x173278ad0>>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

⠹ recording⠸ recording⠼ recording⠴ recording⠦ recording⠧ recording⠇ recording⠏ recording⠋ recording⠙ recording⠹ recording⠸ recording⠼ recording⠴ recording⠦ recording⠧ recording⠇ recording⠏ recording⠋ recording⠙ recording⠹ recording⠸ recording⠼ recording⠴ recording⠦ recording⠧ recording⠇ recording⠏ recording⠋ recording⠙ recording⠹ recording⠸ recording⠼ recording⠴ recording⠦ recording⠧ recording⠇ recording⠏ recording⠋ recording⠙ recording⠹ recording⠸ recording⠼ recording⠴ recording⠦ recording⠧ recording⠇ recording⠏ recording⠋ recording⠙ recording⠹ recording⠸ recording⠼ recording⠴ recording⠦ recording⠧ recording⠇ recording⠏ recording⠋ recording⠙ recording⠹ recording⠸ recording⠼ recording⠴ recording⠦ recording⠧ recording⠇ recording⠏ recording⠋ recording⠙ recording⠹ recording⠸ recording⠼ recording⠴ recording⠦ recording⠧ recording⠇ recording⠏ recording⠋ recording⠙ recording⠹ recording⠸ recording⠼ recording⠴ recording⠦ recording⠧ recording⠇ recording⠏ recording⠋ recording⠙ recording⠹ recordin

In [5]:
import RealtimeSTT
print(dir(RealtimeSTT))


['AudioToTextRecorder', 'AudioToTextRecorderClient', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'audio_recorder', 'audio_recorder_client']


In [9]:
print(dir(recorder))



['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_audio_data_worker', '_check_voice_activity', '_find_tail_match_in_text', '_is_silero_speech', '_is_voice_active', '_is_webrtc_speech', '_on_realtime_transcription_stabilized', '_on_realtime_transcription_update', '_preprocess_output', '_process_wakeword', '_read_stdout', '_realtime_worker', '_recording_worker', '_set_spinner', '_set_state', '_start_thread', '_transcription_worker', 'abort', 'allowed_latency_limit', 'allowed_to_early_transcribe', 'audio', 'audio_buffer', 'audio_interface', 'audio_queue', 'beam_size', 'beam_size_realtime', 'buffer_size', 'clear_audio_queue', 'compute_type', 'debug_mode', 'd

In [2]:
from RealtimeSTT import AudioToTextRecorder
import pyautogui

def process_text(text):
    pyautogui.typewrite(text + " ")

if __name__ == '__main__':
    print("Wait until it says 'speak now'")
    recorder = AudioToTextRecorder()

    while True:
        recorder.text(process_text)

Wait until it says 'speak now'
⠋ speak now⠙ speak now

[2024-11-21 20:38:34.873] [ctranslate2] [thread 27529265] [warning] The compute type inferred from the saved model is float16, but the target device or backend do not support efficient float16 computation. The model weights have been automatically converted to use the float32 compute type instead.


KeyboardInterrupt: 

Error in callback <function Halo.__init__.<locals>.clean_up at 0x313555d00> (for post_run_cell), with arguments args (<ExecutionResult object at 17e800b60, execution_count=2 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 17e8e2660, raw_cell="from RealtimeSTT import AudioToTextRecorder
import.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/wangcan/Desktop/VTT/test.ipynb#X22sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

Error in callback <function Halo.__init__.<locals>.clean_up at 0x313555f80> (for post_run_cell), with arguments args (<ExecutionResult object at 17e800b60, execution_count=2 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 17e8e2660, raw_cell="from RealtimeSTT import AudioToTextRecorder
import.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/wangcan/Desktop/VTT/test.ipynb#X22sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

In [2]:
from RealtimeSTT import AudioToTextRecorder
with AudioToTextRecorder() as recorder:
    print(recorder.text())

⠋ speak now⠙ speak now

[2024-11-26 07:44:33.061] [ctranslate2] [thread 28979891] [warning] The compute type inferred from the saved model is float16, but the target device or backend do not support efficient float16 computation. The model weights have been automatically converted to use the float32 compute type instead.


Hello, hello, can you hear me? Can you hear me?peak now⠧ speak now⠇ recording⠏ recording⠋ recording⠙ recording⠹ recording⠸ recording⠼ recording⠴ recording⠦ recording⠧ recording⠇ recording⠏ recording⠋ recording⠙ recording⠹ recording⠸ recording⠼ recording⠴ recording⠦ recording⠧ recording⠇ recording⠏ recording⠋ recording⠙ recording⠹ recording⠸ recording⠼ recording transcribing⠙ transcribing
RealtimeSTT shutting down
Error in callback <function Halo.__init__.<locals>.clean_up at 0x3095f0400> (for post_run_cell), with arguments args (<ExecutionResult object at 110915b20, execution_count=2 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 110915cd0, raw_cell="from RealtimeSTT import AudioToTextRecorder
with A.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/wangcan/Desktop/VTT/test.ipynb#X23sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

Error in callback <function Halo.__init__.<locals>.clean_up at 0x3095f0180> (for post_run_cell), with arguments args (<ExecutionResult object at 110915b20, execution_count=2 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 110915cd0, raw_cell="from RealtimeSTT import AudioToTextRecorder
with A.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/wangcan/Desktop/VTT/test.ipynb#X23sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given